In [1]:
import pennylane as qml
import numpy as np

dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev, interface="autograd")
def circuit(x, y):
    # prepares the reference state |100>
    qml.BasisState(np.array([1,0,0]), wires=[0,1,2])
    # applies the single excitations
    qml.SingleExcitation(x, wires=[0,1])
    qml.SingleExcitation(y, wires=[0,2])
    return qml.state()

x = -2*np.arcsin(np.sqrt(1/3))
y = -2*np.arcsin(np.sqrt(1/2))
print(circuit(x, y))

[0.        +0.j 0.57735027+0.j 0.57735027+0.j 0.        +0.j
 0.57735027+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


In [2]:
tensor_state = circuit(x, y)
tensor_state

tensor([0.        +0.j, 0.57735027+0.j, 0.57735027+0.j, 0.        +0.j,
        0.57735027+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j], requires_grad=True)

In [3]:
tensor_state.reshape(2,2,2)

tensor([[[0.        +0.j, 0.57735027+0.j],
         [0.57735027+0.j, 0.        +0.j]],

        [[0.57735027+0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j]]], requires_grad=True)

Excitations on a fixed reference state and include only the excitations that preserve the spin orientation of the electron

In [5]:
n_particles = 3
n_qubits = 6

singles, doubles = qml.qchem.excitations(3, 6)
print(f"Single excitations = {singles}")
print(f"Double excitations = {doubles}")

Single excitations = [[0, 4], [1, 3], [1, 5], [2, 4]]
Double excitations = [[0, 1, 3, 4], [0, 1, 4, 5], [1, 2, 3, 4], [1, 2, 4, 5]]


### Double Excitations

In [8]:
dev2 = qml.device('default.qubit', wires=6)

@qml.qnode(dev2, interface="autograd")
def circuit2(x, y):
    # prepares reference state
    qml.BasisState(np.array([1,1,1,0,0,0]), wires=[0,1,2,3,4,5])
    # apply all single excitations
    for i, s in enumerate(singles):
        qml.SingleExcitation(x[i], wires=s)
    # apply all double excitations
    for j, d in enumerate(doubles):
        qml.DoubleExcitation(y[j], wires=d)
    return qml.state()

# random angles of rotation
x = np.random.normal(0, 1, len(singles))
y = np.random.normal(0, 1, len(doubles))

output = circuit2(x, y)

In [9]:
# constructs binary representation of states with non-zero amplitude
states = [np.binary_repr(i, width=6) for i in range(len(output)) if output[i] != 0]
print(states)

['001011', '001110', '011010', '100011', '100110', '101001', '101100', '110010', '111000']


### Controlled Excitation Gates

Single-qubit gates and CNOT gates are universal for quantum computing: they can be used to implement any conceivable quantum computation

In [10]:
dev = qml.device('default.qubit', wires=6)

@qml.qnode(dev, interface="autograd")
def circuit3(x, y, z):
    qml.BasisState(np.array([1, 1, 0, 0, 0, 0]), wires=[i for i in range(6)])
    qml.DoubleExcitation(x, wires=[0, 1, 2, 3])
    qml.DoubleExcitation(y, wires=[0, 1, 4, 5])
    qml.SingleExcitation(z, wires=[1, 3])
    return qml.state()

x = -2 * np.arcsin(np.sqrt(1/4))
y = -2 * np.arcsin(np.sqrt(1/3))
z = -2 * np.arcsin(np.sqrt(1/2))

output = circuit3(x, y, z)
states = [np.binary_repr(i, width=6) for i in range(len(output)) if output[i] != 0]
print(states)

['000011', '001100', '011000', '100100', '110000']


In [11]:
@qml.qnode(dev, interface="autograd")
def circuit4(x, y, z):
    qml.BasisState(np.array([1, 1, 0, 0, 0, 0]), wires=[i for i in range(6)])
    qml.DoubleExcitation(x, wires=[0, 1, 2, 3])
    qml.DoubleExcitation(y, wires=[0, 1, 4, 5])
    # single excitation controlled on qubit 0
    qml.ctrl(qml.SingleExcitation, control=0)(z, wires=[1, 3])
    return qml.state()

output = circuit4(x, y, z)
states = [np.binary_repr(i, width=6) for i in range(len(output)) if output[i] != 0]
print(states)

['000011', '001100', '100100', '110000']
